## Chapter 5　応用

---
### 5.1　機械翻訳


#### 統計翻訳とニューラル翻訳
ある言語の文章を多言語の文章に翻訳する、人手を介さない方法論を機械翻訳（machine translation）と総称する。  
ルールベース翻訳、用例ベース翻訳、統計翻訳（statistical machine translation）などの方法論があり、  
系列変換モデルを利用した機械翻訳はニューラル翻訳（neural machine translation）と呼ばれる場合が多い。

---
#### NMTの典型的なモデル構成
[OpenNMT](http://opennmt.net/)（旧seq2seq-attn）が現在ベースラインと認識されるモデルの代表である。  
この中のモデルの１つについて取り上げる。

符号化器再帰層に２層bi-directional LSTMを使うとする。  
$\Psi(\cdot)$ をLSTMの処理を表す関数とすると、擬似コードは次の通り。

\begin{align*}
\hline{}
&\text{Input: } \, X = (x_{i})_{i=1}^{I} \\
&\text{begin} \\
&\hspace{20pt}  h_{0}^{fw1} \leftarrow 0, h_{0}^{fw2} \leftarrow 0, h_{I+1}^{bw1} \leftarrow 0, h_{I+1}^{bw2} \leftarrow 0 \\
&\hspace{20pt}  \bar{X} \leftarrow E^{s}X \\
&\hspace{20pt}  \text{for} \hspace{5pt} i \leftarrow 1 \hspace{5pt} \text{to} \hspace{5pt} I \hspace{5pt} \text{do} \\
&\hspace{40pt} h_{i}^{fw1} \leftarrow \Psi^{fw1}(\bar{x}_{i},h_{i-1}^{fw1}) \\
&\hspace{40pt} h_{i}^{fw2} \leftarrow \Psi^{fw2}(h_{i}^{fw1},h_{i-1}^{fw2}) \\
&\hspace{20pt} \text{end} \\
&\hspace{20pt} \text{for} \hspace{5pt} i \leftarrow 1 \hspace{5pt} \text{to} \hspace{5pt} I \hspace{5pt} \text{do} \\
&\hspace{40pt} i' \leftarrow I+1-i \\
&\hspace{40pt} h_{i'}^{bw1} \leftarrow \Psi^{bw1}(\bar{x}_{i'},h_{i'+1}^{bw1}) \\
&\hspace{40pt} h_{i'}^{bw2} \leftarrow \Psi^{bw2}(h_{i'}^{bw1},h_{i'+1}^{bw2}) \\
&\hspace{40pt} h_{i'}^{s} = h_{i'}^{fw2} + h_{i'}^{bw2} \\
&\hspace{20pt} \text{end} \\
&\text{end} \\
&\text{Output: } \, H^{s}=(h_{i}^{s})_{i=1}^{I}, h_{I}^{fw1}, h_{I}^{fw2}, h_{1}^{bw1}, h_{1}^{bw2} \\
\hline{}
\end{align*}



次に、復号化再帰層に２層LSTMを使うものとすると、擬似コードは次の通り。

\begin{align*}
\hline{}
&\text{Input: } \, H^{s}=(h_{i})_{i=1}^{I}, h_{I}^{fw1}, h_{I}^{fw2}, h_{1}^{bw1}, h_{1}^{bw2}, Y=(y_{j})_{j=1}^{J} \\
&\text{begin} \\
&\hspace{20pt} z_{0}^{fw1} \leftarrow h_{I}^{fw1} + h_{1}^{bw1} \\
&\hspace{20pt} z_{0}^{fw2} \leftarrow h_{I}^{fw2} + h_{1}^{bw2} \\
&\hspace{20pt} y_{0} \leftarrow y^{(BOS)}, y_{J+1} \leftarrow y^{(EOS)} \\
&\hspace{20pt} c_{0} \leftarrow 0 \\
&\hspace{20pt} \text{for} \hspace{5pt} j \leftarrow 1 \hspace{5pt} \text{to} \hspace{5pt} J+1 \hspace{5pt} \text{do} \\
&\hspace{40pt} \bar{y}_{j} \leftarrow E^{t}y_{j-1} \\
&\hspace{40pt} z_{j}^{0} \leftarrow \operatorname{concat}(c_{j-1},\bar{y}_{j}) \\
&\hspace{40pt} z_{j}^{fw1} \leftarrow \Psi^{fw1}(z_{j}^{0},z_{j-1}^{fw1}) \\
&\hspace{40pt} z_{j}^{fw2} \leftarrow \Psi^{fw2}(z_{j}^{fw1},z_{j-1}^{fw2}) \\
&\hspace{40pt} h_{j}^{t} \leftarrow z_{j}^{fw2} \\
&\hspace{40pt} a_{j} \leftarrow \operatorname{softmax}((H^{s})^{\mathrm{T}}W_{2}^{a}h_{j}^{t}) \\
&\hspace{40pt} c_{j}' \leftarrow H^{s}a_{j} \\
&\hspace{40pt} \tilde{c}_{j} \leftarrow \operatorname{concat}(c'_{j}, h_{j}^{t}) \\
&\hspace{40pt} c_{j} \leftarrow \operatorname{tanh}(W_{1}^{a} \tilde{c}_{j}) \\
&\hspace{20pt} \text{end} \\
&\hspace{20pt} \text{for} \hspace{5pt} j \leftarrow 1 \hspace{5pt} \text{to} \hspace{5pt} J+1 \hspace{5pt} \text{do} \\
&\hspace{40pt} o_{j} \leftarrow \operatorname{softmax}(W^{O}c_{j}) \\
&\hspace{40pt} \text{loss}_{j} \leftarrow \operatorname{nll}(o_{j}, y_{j}) \\
&\hspace{20pt} \text{end} \\
&\hspace{20pt} \text{loss} \, \leftarrow \sum_{j=1}^{J+1} \text{loss}_{j} \\
&\text{end} \\
&\text{Output: } \text{loss} \\
\hline{}
\end{align*}

評価時の復号化処理は一般にビーム探索を用いて処理を行うことが多い。  
学習後に未知データを評価する際の復号化器における復号化処理を擬似コードで表したものは次の通り。  
（これらのアルゴリズム一式はK=1のビーム探索と等価なアルゴリズムである。）

\begin{align*}
\hline{}
&\text{Input: } \, H^{s}=(h_{i})_{i=1}^{I}, h_{I}^{fw1}, h_{I}^{fw2}, h_{1}^{bw1}, h_{1}^{bw2} \\
&\text{begin} \\
&\hspace{20pt} z_{0}^{fw1} \leftarrow h_{I}^{fw1} + h_{1}^{bw1} \\
&\hspace{20pt} z_{0}^{fw2} \leftarrow h_{I}^{fw2} + h_{1}^{bw2} \\
&\hspace{20pt} \hat{y}_{0} \leftarrow y^{(BOS)} \\
&\hspace{20pt} c_{0} \leftarrow 0 \\
&\hspace{20pt} j=0 \\
&\hspace{20pt} \text{repeat} \\
&\hspace{40pt} j \leftarrow j+1 \\
&\hspace{40pt} \bar{y}_{j} \leftarrow E^{t}\hat{y}_{j-1} \\
&\hspace{40pt} z_{j}^{0} \leftarrow \operatorname{concat}(c_{j-1},\bar{y}_{j}) \\
&\hspace{40pt} z_{j}^{fw1} \leftarrow \Psi^{fw1}(z_{j}^{0},z_{j-1}^{fw1}) \\
&\hspace{40pt} z_{j}^{fw2} \leftarrow \Psi^{fw2}(z_{j}^{fw1},z_{j-1}^{fw2}) \\
&\hspace{40pt} h_{j}^{t} \leftarrow z_{j}^{fw2} \\
&\hspace{40pt} a_{j} \leftarrow \operatorname{softmax}((H^{s})^{\mathrm{T}}W_{2}^{a}h_{j}^{t}) \\
&\hspace{40pt} c_{j}' \leftarrow H^{s}a_{j} \\
&\hspace{40pt} \tilde{c}_{j} \leftarrow \operatorname{concat}(c'_{j}, h_{j}^{t}) \\
&\hspace{40pt} c_{j} \leftarrow \operatorname{tanh}(W_{1}^{a} \tilde{c}_{j}) \\
&\hspace{40pt} o_{j} \leftarrow W^{O}c_{j} \\
&\hspace{40pt} \hat{y}_{i} \leftarrow \sigma'_{m}(o_{j}) \\
&\hspace{20pt} \text{until} \hspace{5pt} \hat{y}_{j}=y^{(EOS)} \\
&\hspace{20pt} J \leftarrow j-1 \\
&\text{end} \\
&\text{Output: } \hat{Y} = (\hat{y}_{j})_{j=1}^{J} \\
\hline{}
\end{align*}

---
#### 入出力の処理単位、未知語に対する改良
